In [12]:
# import libraries
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid

from scipy.integrate import odeint, solve_ivp
from scipy.fft import fft


import shap as shap
try:
    import lime
    import lime.lime_tabular    
except ImportError:
    pass

# Machine Learning Libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.utils import Sequence
from tensorflow import keras

# for reproducibility of this notebook:
rng = np.random.RandomState(42)
tf.random.set_seed(42)
np.random.seed(42)


import os
import sys


#idx = int(os.environ["SLURM_ARRAY_TASK_ID"])

#feature_setting = str(os.environ["Setting"])

#model_setting = str(os.environ["Model"])
idx = 0
model_setting = "True"
feature_setting = "Gamma"

"""
Define Parameter Configuration to Model

    Parameters
    ----------
    alpha : float, linear stiffness
    beta  : float, non linearity in the restoring force
    gamma : float, amplitude of the periodic driving force
    delta : float, amount of damping
    omega : float, angular frequency of the periodic driving force
"""   

parameter_list = [{'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2}, 
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 1.0, 'omega' : 1.2},
                  {'alpha' : 1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2}, 
                  {'alpha' : -1.0, 'beta' : -1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 0.1},
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.5, 'delta' : 0.3, 'omega' : 1.2}]

"""[{'alpha' : 1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2}, 
                  {'alpha' : 1.0, 'beta' : -0.5, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : 1.0, 'beta' : -0.5, 'gamma' : 0.37, 'delta' : 1.0, 'omega' : 1.2}, 
                  {'alpha' : 1.0, 'beta' : -0.5, 'gamma' : 0.5, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : 1.0, 'beta' : -0.5, 'gamma' : 0.37, 'delta' : 0.0, 'omega' : 1.2},
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.37, 'delta' : 1.0, 'omega' : 1.2}, 
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.5, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : -1.0, 'beta' : 1.0, 'gamma' : 0.0, 'delta' : 0.3, 'omega' : 0.0},
                  {'alpha' : -1.0, 'beta' : -1.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2},
                  {'alpha' : 0.0, 'beta' : 0.0, 'gamma' : 0.37, 'delta' : 0.3, 'omega' : 1.2}]
"""

dict_param = parameter_list[idx]

from OtherFunctions import *

if feature_setting == "Base":
    num_samples_ml = 100000
    from  BaseDuffing import Duffing
elif feature_setting == "Random":
    num_samples_ml = 100000
    from  RandomDuffing import Duffing
elif feature_setting == "Energy":
    num_samples_ml = 100000
    from  EnergyDuffing import Duffing
elif feature_setting == "Gamma":
    num_samples_ml = 10
    from  GammaDuffing import Duffing


    


if __name__ == '__main__':
    duffing = Duffing(parameters = dict_param)
    eom = duffing.eom
    suffix = feature_setting + "_" + model_setting + "_" + duffing.suffix

    end_time = 100
    duffing.generate(num_samples_ml, samples = 100, end_time = end_time) #samples prev 100
    duffing.scale_features()
    X_train, X_test, y_train, y_test = train_test_split(duffing.X_df[duffing.features], 
                                                        duffing.X_df[duffing.labels], test_size=0.1, random_state=42)
    
    X = X_test
    y = y_test
    
    # Create a basic model instance
    if model_setting == "Complex":
        model = MLModel()
    elif model_setting == "Simple":
        model = SimpleModel()
    elif model_setting == "True":
        model = duffing
        
    
    if (model_setting == "Simple") or (model_setting == "Complex"):
        """
        Train Model
        """
        callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss',patience=3),
                     tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)]


        history=model.fit(X_train, y_train, steps_per_epoch=None, epochs=500, validation_split=0.2, 
                          batch_size=1024, shuffle=True, callbacks=callbacks, verbose=0)


        model.save('Models/Model'+suffix)
        with open('Models/TrainingHistory/'+suffix, 'wb') as file_pi:
            pickle.dump(history.history, file_pi)
            
    if model_setting == "Complex":
        model_ = model
    elif model_setting == "Simple":
        model_ = model
    elif model_setting == "True":
        model_ = duffing.predict
    
    def lime_x(X):
        return model.predict(X)[:,0]
    def lime_v(X):
        return model.predict(X)[:,1]
    

    explainers = ["kernel", "sampling", "lime", "numeric"]
    lime_models = [lime_x, lime_v]

    background = shap.sample(X_test, 100)
    choice = X_test.iloc[np.sort(np.random.choice(X_test.shape[0], 100, replace =False))]

    
    big_df = pd.DataFrame()
    for explainer in explainers:
        if explainer == "kernel":
            temp_explainer = shap.KernelExplainer(model_, background)
            temp_vals = temp_explainer.shap_values(choice)
        elif explainer == "sampling":
            temp_explainer = shap.SamplingExplainer(model_, background)
            temp_vals = temp_explainer.shap_values(choice)
        elif explainer == "lime":
            temp_explainer = MyLime(lime_models, X_test, mode='regression')
            temp_vals = temp_explainer.attributions(choice)
        elif explainer == "numeric":
            temp_explainer = NumericExplainer(model, duffing.features, duffing.labels, h = 0.001)
            temp_vals = temp_explainer.feature_att(choice)
        else:
            print("not a valid explainer type")
        big_df = big_df.append(duffing.vals_to_df(temp_vals, choice, explainer = explainer, suffix = suffix))


    big_df.to_csv("Results/explainer_dataframe_"+suffix+".csv")  



Generating Data…: 100%|████████████████████| 10/10 [00:32<00:00,  3.26s/it]

Provided model function fails when applied to the provided data set.


TypeError: '(slice(0, 0, None), slice(None, None, None))' is an invalid key

In [10]:
duffing.X_df

,x0,v0,t,gamma,xt,vt
0,0.198781,0.712857,0.000000,0.0,-1.204630,0.845368
1,0.198781,0.712857,0.010101,0.0,-0.493528,0.462093
2,0.198781,0.712857,0.020202,0.0,-0.238048,0.093340
3,0.198781,0.712857,0.030303,0.0,-0.254758,-0.118608
4,0.198781,0.712857,0.040404,0.0,-0.484140,-0.351698
...,...,...,...,...,...,...
99995,0.017724,0.126249,0.959596,1.0,1.999520,-0.598644
99996,0.017724,0.126249,0.969697,1.0,0.376159,-1.662814
99997,0.017724,0.126249,0.979798,1.0,-1.430924,-1.934316
99998,0.017724,0.126249,0.989899,1.0,-1.486254,1.810167


In [13]:
duffing.predict(duffing.X_df[duffing.features])

TypeError: '(slice(0, 0, None), slice(None, None, None))' is an invalid key

In [6]:
temp_explainer = shap.KernelExplainer(model_, background)
temp_vals = temp_explainer.shap_values(choice)

Provided model function fails when applied to the provided data set.


TypeError: '(slice(0, 0, None), slice(None, None, None))' is an invalid key

In [28]:
    def predict(X, scaler):
        if type(X) == pd.core.frame.DataFrame:
            X_temp = pd.DataFrame(scaler.inverse_transform(X.values), columns=X.columns)
        elif type(X) == pd.core.series.Series:
            X_temp = pd.DataFrame(scaler.inverse_transform(X.values.reshape(1,-1)), columns=X.index)
        else:
            X_temp = pd.DataFrame(scaler.inverse_transform(X), columns=self.features)
        y = np.ones((np.shape(X_temp)[0], 2))
        #for i in range(X_temp.shape[0]):
            #X_temp['gamma']
            #duffing.parameters["gamma"] = X_temp['gamma'][i]
            #traj = solve_ivp(self.eom, [0, X_temp['t'].iloc[i]], [X_temp['x0'].iloc[i], X_temp['v0'].iloc[i]], 
            #                t_eval = None, events = [self.termination_event])
            #y[i] = [traj.y[0][-1], traj.y[1][-1]]
        return X_temp

In [31]:
predict(duffing.X_df[duffing.features], scaler = duffing.scaler)

,x0,v0,t,gamma
0,-1.204630,0.845368,0.0,0.0
1,-1.204630,0.845368,1.0,0.0
2,-1.204630,0.845368,2.0,0.0
3,-1.204630,0.845368,3.0,0.0
4,-1.204630,0.845368,4.0,0.0
...,...,...,...,...
99995,-1.928217,-1.494985,95.0,1.0
99996,-1.928217,-1.494985,96.0,1.0
99997,-1.928217,-1.494985,97.0,1.0
99998,-1.928217,-1.494985,98.0,1.0


In [35]:
duffing.X_df[duffing.features].loc[100:100,:]

,x0,v0,t,gamma
100,0.790637,0.607201,0.0,0.010101
